# Periodisch angetriebenes Pendel

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

In diesem Jupyter-Notebook wird das gedämpfte mathematische Pendel unter dem Einfluss eines harmonischen Antriebs untersucht. Zunächst wird die zugehörige Bewegungsgleichung in dimensionsloser Form

$$\ddot\theta + k \dot\theta + \sin(\theta) = F \cos(\Omega t)%$$

gelöst. Im zweiten Teil wird dann die Periode des Attraktors als Funktion der Antriebsstärke $F$ für feste Werte der dimensionslosen Dämpfungsstärke $k=1/2$ und der dimensionslosen Antriebsfrequenz $\Omega=2/3$ bestimmt. Die dabei auftretende Periodenverdoppelung wird abschließend in einem Bifurkationsdiagramm veranschaulicht. 

## Auslenkung als Funktion der Zeit

### Importanweisungen

Zu den bereits bekannten Importanweisungen kommt hier der Import der Funktionen `clear_output` und `display` aus `IPython.display` hinzu. Diese beiden Funktionen werden zum schrittweisen Aufbau des Bifurkationsdiagramms im letzten Teil dieses Jupyter-Notebooks verwendet.

In [ ]:
from math import cos, pi, sin
import numpy as np
from scipy import integrate
from IPython.display import clear_output, display
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt

plt.style.use("numphyspy.style")

### Implementierung des Differentialgleichungssystems

Die Bewegungsgleichung wird durch zwei gekoppelte Differentialgleichungen 1. Ordnung mit drei Parametern dargestellt, der Dämpfungsstärke `k`, der Antriebsstärke `f` und der Antriebsfrequenz `omega`.

In [ ]:
def dx_dt(t, x, k, f, omega):
    v = x[1]
    a = f*cos(omega*t)-k*x[1]-sin(x[0])
    return v, a

### Lösung des Differentialgleichungssystems

Die Bewegungsgleichung wird mit Hilfe der SciPy-Funktion `integrate.solve_ivp` gelöst, wobei ein Runge-Kutta-Verfahren der Ordnung 8 verwendet wird.

In [ ]:
def trajectory(t_end, n_out, k, f, omega, x_0, v_0):
    t = np.linspace(0, t_end, n_out)
    solution = integrate.solve_ivp(
        dx_dt, (0, t_end), [x_0, v_0], t_eval=t,
        args=(k, f, omega), method="DOP853",
        atol=1e-10, rtol=1e-10)
    return t, solution.y[0, :]

### Implementierung der Bedienelemente und graphische Darstellung der Zeitabhängigkeit der Schwingungsamplitude

Mit Hilfe der Schieberegler lassen sich die folgenden Parameter einstellen:
- `t_end`: Länge des zu betrachtenden Zeitintervalls
- `f`: dimensionslose Antriebsstärke
- `theta_0`: Anfangsauslenkung
- `v_theta_0`: anfängliche Winkelgeschwindigkeit

Da uns hier vor allem die Abhängigkeit von der Antriebsstärke interessiert, sind die Werte für die dimensionslose Dämpfungsstärke und die dimensionslose Antriebsfrequenz in der Funktion `plot_dynamics` festgelegt und können dort bei Bedarf angepasst werden. Zudem wird dort die Zahl `n_out` der Ausgabepunkte fest vorgegeben.

In [ ]:
widget_dict = {"t_end":
               widgets.FloatSlider(
                   value=100, min=100, max=500, step=100,
                   description=r"$\tau_\text{end}$"),
               "f":
               widgets.FloatSlider(
                   value=1.05, min=1.05, max=1.09,
                   step=0.001, readout_format=".3f",
                   description="$F$"),
               "theta_0":
               widgets.FloatSlider(
                   value=0.5, min=-1, max=1, step=0.1,
                   description=r"$\theta(0)$"),
               "v_theta_0":
               widgets.FloatSlider(
                   value=0, min=-2.5, max=2.5, step=0.1,
                   description=r"$v_\theta(0)$")
               }

@interact(**widget_dict)
def plot_dynamics(t_end, f, theta_0, v_theta_0):
    omega = 2/3
    k = 0.5
    n_out = 1000
    t, theta = trajectory(t_end, n_out, k, f, omega,
                          theta_0, v_theta_0)

    fig, ax = plt.subplots()
    ax.plot(t, theta)
    ax.set_xlabel("$t$")
    ax.set_ylabel(r"$\theta$")

## Periodendauer als Funktion der Antriebsstärke

### Berechnung der Periodendauer als Funktion der Antriebsstärke

Für `n_f` Werte der Antriebsstärke zwischen `f_min` und `f_max` wird die Periodendauer in Einheiten der Anregungsperiode `t_period` berechnet.

Innerhalb der Funktion wird ein Startpunkt `x_0, v_0` im Phasenraum festgelegt. Um von dort möglichst nahe an den Attraktor für die minimiale Antriebsstärke zu gelangen, wird zunächst eine Propagation über `n_initial` Antriebsperioden durchgeführt.

In der folgenden Schleife über die Antriebsstärken werden zunächst die Parameter für die folgenden Integrationen gesetzt. Die anschließende Propagation über `n_between` Anregungsperioden soll dafür sorgen, dass der Attraktor für die neue Antriebsstärke möglichst gut erreicht wird. `n_between` kann dabei kleiner als `n_initial`gewählt werden.

Nun beginnt die eigentliche Bestimmung der Periode, in der jeweils über eine Anregungsperiode integriert wird und untersucht wird, ob der Endpunkt im Phasenraum hinreichend nahe am Anfangspunkt liegt. Da es hier passieren kann, dass die `while`-Schleife nicht beendet wird, wird die Bestimmung der Periode bei einer maximalen Anzahl von Perioden `MAX_PERIOD` für die betreffende Antriebsstärke ergebnislos abgebrochen. Der betreffende Wert wird zu Beginn der folgenden Codezelle gesetzt und kann dort bei Bedarf angepasst werden. 

In [ ]:
MAX_PERIOD = 8

def period(n_initial, n_between, f_min, f_max, n_f,
           k, omega, eps):
    t_period = 2*pi / omega
    x_0, v_0 = 0.5, 0
    f_values = []
    np_values = []

    t_end = n_initial * t_period
    solution = integrate.solve_ivp(
        dx_dt, (0, t_end), [x_0, v_0],
        args=(f_min, k, omega), method="DOP853",
        atol=1.e-10, rtol=1e-10)
    x_0, v_0 = solution.y[:, -1]

    for f in np.linspace(f_min, f_max, n_f):
        kwargs = {"args": (k, f, omega),
                  "method": "DOP853",
                  "atol": 1e-10, "rtol": 1e-10}

        t_end = n_between * t_period
        solution = integrate.solve_ivp(
            dx_dt, (0, t_end), [x_0, v_0], **kwargs)
        x_0, v_0 = solution.y[:, -1]

        first_run = True
        n_period = 0
        x_1, v_1 = x_0, v_0
        while first_run or (x_1-x_0)**2+(v_1-v_0)**2 > eps:
            first_run = False
            solution = integrate.solve_ivp(
                dx_dt, (0, t_period), [x_1, v_1], **kwargs)
            x_1, v_1 = solution.y[:, -1]
            n_period = n_period + 1
            if n_period > MAX_PERIOD:
                break
        else:
            f_values.append(f)
            np_values.append(n_period)
            print(f"{f:.4f} {n_period:5}", end="\r")
    return f_values, np_values

### Implementierung der Bedienelemente und graphische Darstellung der Periodenverdopplung

Mit Hilfe der Schieberegler lassen sich die folgenden Parameter einstellen:
- `n_initial`: Anzahl der Anregungsperioden, um für die geringste Antriebsstärke einen Punkt auf dem Attraktor zu erreichen
- `n_between`: Anzahl der Anregungsperioden, um für die nächste Antriebsstärke einen Punkt auf dem Attraktor zu erreichen
- `n_f`: Anzahl der zu betrachtenden Antriebsstärken
- `eps`: Abstandsschwelle im Phasenraum zur Detektion der Periode

In der Funktion `plot_period_doubling` werden die dimensionslose Anregungsfrequenz `omega` und die dimensionslose Dämpfungsstärke `k` festgelegt. Bei Bedarf können sie dort geändert werden. Zudem wird ein passendes Intervall für die Antriebsstärke zwischen `f_min` und `f_max` fest vorgegeben.

Da die Berechnung der Periodenwerte einige Zeit in Anspruch nimmt, muss sie jeweils nach Einstellung neuer Parameter explizit gestartet werden.

In [ ]:
widget_dict = {"n_initial":
               widgets.IntSlider(
                   value=1000, min=100, max=2000, step=100,
                   description=r"$n_\text{init}$"),
               "n_between":
               widgets.IntSlider(
                   value=500, min=100, max=2000, step=100,
                   description=r"$n_\text{between}$"),
               "n_f":
               widgets.IntSlider(
                   value=50, min=10, max=200, step=10,
                   description="$N_F$"),
               "eps":
               widgets.FloatLogSlider(
                   value=1e-10, min=-16, max=-6, step=1,
                   description=r"$\varepsilon$")
               }

interact_start = interact_manual.options(
    manual_name="Start Berechnung")

@interact_start(**widget_dict)
def plot_period_doubling(n_initial, n_between, n_f, eps):
    omega = 2/3
    k = 0.5
    f_min, f_max = 1.05, 1.09
    f_values, np_values = period(
        n_initial, n_between, f_min, f_max, n_f, k, omega,
        eps)

    fig, ax = plt.subplots()
    ax.plot(f_values, np_values, linestyle="None",
            marker="o", markersize=1)
    ax.set_xlabel("$F$")
    ax.set_ylabel(r"$(\Omega/2\pi)T$")

## Bifurkationsdiagramm

### Propagation zum Attraktor

Ausgehend von einem beliebigen Anfangszustand `x_0, v_0` im Phasenraum wird über eine große Zahl `n_initial` von Anregungsperioden propagiert, um möglichst gut den Attraktor zu erreichen. Dadurch erhält man den Anfangszustand für die weitere Rechnung.

In [ ]:
def initial_attractor(n_initial, k, f, omega, x0, v0):
    t_initial = 2*pi/omega * n_initial
    solution = integrate.solve_ivp(
        dx_dt, (0, t_initial), [x0, v0],
        args=(k, f, omega), method="DOP853",
        atol=1e-10, rtol=1e-10)
    return solution.y[:, -1]

### Berechnung der Punkte für das Bifurkationsdiagramm

Nachdem mit Hilfe der Funktion `initial_attractor` ein Punkt im Phasenraum bestimmt wurde, der hinreichend nahe am Attraktor liegt, werden nun `n_display` Phasenraumpunkte berechnet, die jeweils einen zeitlichen Abstand von einer Anregungsperiode besitzen. Neben den Werten der Pendelauslenkungen wird der Endpunkt im Phasenraum zurückgegeben, der für die folgende Rechnung mit einer anderen Antriebsstärke verwendet wird.

In [ ]:
def bifurcation_points(n_display, k, f, omega, x0, v0):
    x0, v0 = initial_attractor(n_display, k, f, omega,
                               x0, v0)
    x_values = []
    for n in range(n_display):
        solution = integrate.solve_ivp(
            dx_dt, (0, 2*pi/omega), [x0, v0],
            args=(k, f, omega), method="DOP853",
            atol=1e-10, rtol=1e-10)
        x0, v0 = solution.y[:, -1]
        x_values.append(x0)
    return x_values, x0, v0

### Implementierung der Bedienelemente und graphische Darstellung des Bifurkationsdiagramms

Mit Hilfe der Schieberegler lassen sich die folgenden Parameter einstellen:
- `n_initial`: Anzahl der Antriebsperioden zur Erreichung des Attraktors für die minimale Antriebsstärke
- `n_display`: Anzahl der darzustellenden Auslenkungen im Abstand von jeweils einer Anregungsperiode
- `omega`: dimensionslose Anregungsfrequenz
- `k`: dimensionslose Dämpfungsstärke
- `f_min`: minimale dimensionslose Antriebsstärke
- `f_max`: maximale dimensionslose Antriebsstärke
- `n_f`: Anzahl der zu betrachtenden Antriebsstärken

Die Parameter für den zu `omega` gehörigen Schieberegler sind so gewählt, dass der Wert 2/3 exakt eingestellt werden kann.

Zu Beginn der Funktion `plot_bifurcation_diagramm` wird sichergestellt, dass `f_min` tatsächlich der minimalen Antriebsstärke entspricht. Anschließend werden für `n_f` Antriebsstärken zwischen `f_min` und `f_max` jeweils `n_display` Auslenkungen im Abstand einer Anregungsperiode berechnet und dargestellt. Da die Rechnung relativ zeitaufwändig ist, wird die graphische Darstellung schrittweise aufgebaut, um die Veränderung der Periodendauern schon während der Berechnung mitverfolgen zu können.

In [ ]:
widget_dict = {"n_initial":
               widgets.IntSlider(
                   value=3000, min=1000, max=10000,
                   step=1000,
                   description=r"$n_\text{initial}$"),
               "n_display":
               widgets.IntSlider(
                   value=300, min=100, max=1000, step=100,
                   description=r"$n_\text{display}$"),
               "omega":
               widgets.FloatSlider(
                   value=2/3, min=0.5, max=1, step=1/90,
                   description=r"$\Omega$"),
               "k":
               widgets.FloatSlider(
                   value=0.5, min=0.1, max=1, step=0.1,
                   description="$k$"),
               "f_min":
               widgets.FloatSlider(
                   value=1.05, min=0.5, max=2, step=0.01,
                   description=r"$F_\text{min}$"),
               "f_max":
               widgets.FloatSlider(
                   value=1.09, min=0.5, max=2, step=0.01,
                   description=r"$F_\text{max}$"),
               "n_f":
               widgets.IntSlider(
                   value=300, min=100, max=1000, step=100,
                   description="$n_F$")
               }

@interact_start(**widget_dict)
def plot_bifurcation_diagramm(n_initial, n_display, omega,
                              k, f_min, f_max, n_f):
    if f_min > f_max:
        f_min, f_max = f_max, f_min
    x0, v0 = 0.5, 0
    x0, v0 = initial_attractor(n_initial, k, f_min, omega,
                               x0, v0)
    fvals = []
    xvals = []
    fig, ax = plt.subplots()
    ax.set_xlim(f_min, f_max)
    ax.set_xlabel("$F$")
    ax.set_ylabel(r"$\theta$")

    for f in np.linspace(f_min, f_max, n_f):
        x_values, x0, v0 = bifurcation_points(
            n_display, k, f, omega, x0, v0)
        xvals.extend(x_values)
        fvals.extend(f*np.ones_like(x_values))
        ax.plot(fvals, xvals, linestyle="None", color="blue",
                marker="o", markersize=0.1)
        display(fig)
        clear_output(wait=True)